1)
a) Complexité de l'évaluation des LLMs vs logiciels traditionnels :

- Les LLMs produisent des sorties textuelles ouvertes et créatives
- Il n'y a pas de "bonne réponse" unique comme référence
- Le contexte et la nuance sont essentiels
- La qualité est subjective et dépend de multiples facteurs


b) Raisons d'évaluer la sécurité des LLMs :

- Prévention des biais et discriminations
- Protection contre les contenus toxiques/dangereux
- Vérification de la fiabilité des informations
- Respect de la confidentialité des données


c) Contribution des tests adversariaux :

- Identification des faiblesses et vulnérabilités
- Amélioration de la robustesse
- Test des limites du modèle
- Développement de meilleures protections


d) Limitations des métriques automatiques vs évaluation humaine :

- Les métriques automatiques ne captent pas la nuance
- L'évaluation humaine est plus contextuelle mais subjective
- Les métriques automatiques sont rapides mais superficielles
- Besoin d'une approche hybride

In [6]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize

# Création d'un objet de lissage
smoother = SmoothingFunction()

# Téléchargement des ressources NLTK nécessaires
# nltk.download('punkt')

# Création des phrases de référence et candidate
reference_text = "Despite the increasing reliance on artificial intelligence in various industries, human oversight remains essential to ensure ethical and effective implementation."
candidate_text = "Although AI is being used more in industries, human supervision is still necessary for ethical and effective application."

# Tokenization - conversion des phrases en listes de mots
# Nous créons une liste de références
reference = [word_tokenize(reference_text.lower())]  # On met en minuscules pour standardiser
candidate = word_tokenize(candidate_text.lower())

# Affichage des tokens pour vérification
print("Tokens de référence:", reference[0])
print("Tokens candidats:", candidate)

# Calcul du score BLEU
# Par défaut, utilise les poids uniformes pour les n-grammes (1-gram à 4-gram)
bleu_score = sentence_bleu(reference, candidate)

# Calcul des scores BLEU pour différents n-grammes
bleu_1 = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))  # Uniquement 1-gram
bleu_2 = sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0))  # 1-gram et 2-gram
bleu_3 = sentence_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0))  # 1-gram à 3-gram
bleu_4 = sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25))  # 1-gram à 4-gram

print(f"\nScores BLEU:")
print(f"BLEU-1: {bleu_1:.4f}")
print(f"BLEU-2: {bleu_2:.4f}")
print(f"BLEU-3: {bleu_3:.4f}")
print(f"BLEU-4: {bleu_4:.4f}")


Tokens de référence: ['despite', 'the', 'increasing', 'reliance', 'on', 'artificial', 'intelligence', 'in', 'various', 'industries', ',', 'human', 'oversight', 'remains', 'essential', 'to', 'ensure', 'ethical', 'and', 'effective', 'implementation', '.']
Tokens candidats: ['although', 'ai', 'is', 'being', 'used', 'more', 'in', 'industries', ',', 'human', 'supervision', 'is', 'still', 'necessary', 'for', 'ethical', 'and', 'effective', 'application', '.']

Scores BLEU:
BLEU-1: 0.3619
BLEU-2: 0.2626
BLEU-3: 0.1937
BLEU-4: 0.0000


/opt/anaconda3/envs/tf_m1/lib/python3.9/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [7]:
pip install rouge-score


Note: you may need to restart the kernel to use updated packages.


In [8]:
from rouge_score import rouge_scorer

# Textes de référence et généré
reference = "In the face of rapid climate change, global initiatives must focus on reducing carbon emissions and developing sustainable energy sources to mitigate environmental impact."
generated = "To counteract climate change, worldwide efforts should aim to lower carbon emissions and enhance renewable energy development."

# Création du scorer avec différentes métriques ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Calcul des scores
scores = scorer.score(reference, generated)

# Affichage des résultats
print("Scores ROUGE :")
for metric, score in scores.items():
    print(f"{metric}:")
    print(f"  Précision: {score.precision:.4f}")
    print(f"  Rappel: {score.recall:.4f}")
    print(f"  F1: {score.fmeasure:.4f}")



Scores ROUGE :
rouge1:
  Précision: 0.4706
  Rappel: 0.3333
  F1: 0.3902
rouge2:
  Précision: 0.1875
  Rappel: 0.1304
  F1: 0.1538
rougeL:
  Précision: 0.3529
  Rappel: 0.2500
  F1: 0.2927


2) Limitations Révélées par nos Scores :
a) Problème de Synonymes :
BLEU-1 (0.3619) et ROUGE-1 (F1: 0.3902) montrent des scores moyens car ils ne reconnaissent pas correctement les équivalences :
- "oversight" vs "supervision"
- "remains essential" vs "is still necessary"
- "artificial intelligence" vs "AI"

b) Structure Phrastique :
- La chute drastique de BLEU-4 (0.0000) montre que la métrique pénalise trop les reformulations
- ROUGE-L (F1: 0.2927) indique une différence significative dans la structure, même si le sens est préservé

c) Contexte :
- Les deux métriques ignorent que "implementation" et "application" sont contextuellement équivalents
- Les scores bas ne reflètent pas la préservation du sens global

In [9]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from collections import Counter

# Structure plus robuste
class AdvancedLLMEvaluator:
    def __init__(self):
        self.smoother = SmoothingFunction()
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Gestion Optimisée des Calculs
    def calculate_metrics(self, reference, candidate):
        """Calcul complet des métriques d'évaluation"""
        results = {
            'bleu': self.calculate_bleu(reference, candidate),
            'rouge': self.calculate_rouge(reference, candidate),
            'additional': self.calculate_additional_metrics(reference, candidate)
        }
        return results

    def calculate_bleu(self, reference, candidate):
        """Calcul des scores BLEU avec différents n-grammes"""
        reference_tokens = [word_tokenize(reference.lower())]
        candidate_tokens = word_tokenize(candidate.lower())
        
        return {
            'bleu-1': sentence_bleu(reference_tokens, candidate_tokens, 
                                  weights=(1, 0, 0, 0), 
                                  smoothing_function=self.smoother.method1),
            'bleu-2': sentence_bleu(reference_tokens, candidate_tokens, 
                                  weights=(0.5, 0.5, 0, 0),
                                  smoothing_function=self.smoother.method1),
            'bleu-3': sentence_bleu(reference_tokens, candidate_tokens, 
                                  weights=(0.33, 0.33, 0.33, 0),
                                  smoothing_function=self.smoother.method1),
            'bleu-4': sentence_bleu(reference_tokens, candidate_tokens, 
                                  weights=(0.25, 0.25, 0.25, 0.25),
                                  smoothing_function=self.smoother.method1)
        }

    def calculate_rouge(self, reference, candidate):
        """Calcul des scores ROUGE"""
        return self.rouge_scorer.score(reference, candidate)

    # Métriques Supplémentaires
    def calculate_additional_metrics(self, reference, candidate):
        """Calcul de métriques supplémentaires"""
        ref_tokens = word_tokenize(reference.lower())
        cand_tokens = word_tokenize(candidate.lower())
        
        return {
            'lexical_diversity': {
                'reference': len(set(ref_tokens)) / len(ref_tokens),
                'candidate': len(set(cand_tokens)) / len(cand_tokens)
            },
            'length_ratio': min(len(cand_tokens), len(ref_tokens)) / max(len(cand_tokens), len(ref_tokens))
        }

def evaluate_llm_outputs():
    """Fonction principale d'évaluation"""
    # Exemples de l'exercice
    examples = [
        {
            'name': "Exemple AI",
            'reference': "Despite the increasing reliance on artificial intelligence in various industries, human oversight remains essential to ensure ethical and effective implementation.",
            'candidate': "Although AI is being used more in industries, human supervision is still necessary for ethical and effective application."
        },
        {
            'name': "Exemple Climate Change",
            'reference': "In the face of rapid climate change, global initiatives must focus on reducing carbon emissions and developing sustainable energy sources to mitigate environmental impact.",
            'candidate': "To counteract climate change, worldwide efforts should aim to lower carbon emissions and enhance renewable energy development."
        }
    ]

    evaluator = AdvancedLLMEvaluator()

    for example in examples:
        print(f"\n{example['name']}:")
        results = evaluator.calculate_metrics(example['reference'], example['candidate'])
        
        # Affichage des scores BLEU
        print("\nScores BLEU:")
        for metric, score in results['bleu'].items():
            print(f"  {metric}: {score:.4f}")
        
        # Affichage des scores ROUGE
        print("\nScores ROUGE:")
        for metric, score in results['rouge'].items():
            print(f"  {metric}:")
            print(f"    Précision: {score.precision:.4f}")
            print(f"    Rappel: {score.recall:.4f}")
            print(f"    F1: {score.fmeasure:.4f}")
        
        # Affichage des métriques supplémentaires
        print("\nMétriques supplémentaires:")
        add_metrics = results['additional']
        print(f"  Diversité lexicale (référence): {add_metrics['lexical_diversity']['reference']:.4f}")
        print(f"  Diversité lexicale (candidat): {add_metrics['lexical_diversity']['candidate']:.4f}")
        print(f"  Ratio de longueur: {add_metrics['length_ratio']:.4f}")

# Gestion des Erreurs Plus Efficace
if __name__ == "__main__":
    try:
        nltk.download('punkt')  # Téléchargement des ressources nécessaires
        evaluate_llm_outputs()
    except Exception as e:
        print(f"Erreur lors de l'évaluation : {e}")




Exemple AI:

Scores BLEU:
  bleu-1: 0.3619
  bleu-2: 0.2626
  bleu-3: 0.1937
  bleu-4: 0.0779

Scores ROUGE:
  rouge1:
    Précision: 0.3333
    Rappel: 0.3000
    F1: 0.3158
  rouge2:
    Précision: 0.1765
    Rappel: 0.1579
    F1: 0.1667
  rougeL:
    Précision: 0.3333
    Rappel: 0.3000
    F1: 0.3158

Métriques supplémentaires:
  Diversité lexicale (référence): 1.0000
  Diversité lexicale (candidat): 0.9500
  Ratio de longueur: 0.9091

Exemple Climate Change:

Scores BLEU:
  bleu-1: 0.3277
  bleu-2: 0.2245
  bleu-3: 0.1624
  bleu-4: 0.0649

Scores ROUGE:
  rouge1:
    Précision: 0.4706
    Rappel: 0.3333
    F1: 0.3902
  rouge2:
    Précision: 0.1875
    Rappel: 0.1304
    F1: 0.1538
  rougeL:
    Précision: 0.3529
    Rappel: 0.2500
    F1: 0.2927

Métriques supplémentaires:
  Diversité lexicale (référence): 1.0000
  Diversité lexicale (candidat): 0.9474
  Ratio de longueur: 0.7308


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ludovicveltz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
def calculate_perplexity(probability):
    return 1 / probability

# Comparaison des modèles
model_A_perp = calculate_perplexity(0.8)
model_B_perp = calculate_perplexity(0.4)

print(f"Perplexité Modèle A: {model_A_perp:.2f}")
print(f"Perplexité Modèle B: {model_B_perp:.2f}")


Perplexité Modèle A: 1.25
Perplexité Modèle B: 2.50


Comparaison des Modèles A et B :
3) Calcul de la perplexité :

Formule : Perplexité = 1/P(w), où P(w) est la probabilité du mot
- Modèle A : Perplexité = 1/0.8 = 1.25
- Modèle B : Perplexité = 1/0.4 = 2.5

Analyse :
- Le Modèle A a une perplexité plus faible (1.25 < 2.5)
- Une perplexité plus basse indique une meilleure confiance dans la prédiction
- Le Modèle A est donc plus performant pour ce mot spécifique

Analyse du Modèle avec Perplexité de 100 :

Implications de Performance :
- Une perplexité de 100 signifie que le modèle est incertain dans ses prédictions
- En moyenne, le modèle hésite entre 100 choix possibles pour chaque token
- Cette valeur suggère une performance moyenne à faible selon le contexte

Causes Possibles :
- Données d'entraînement insuffisantes ou non représentatives
- Complexité excessive du domaine
- Architecture du modèle non optimale
- Hyperparamètres mal ajustés

Suggestions d'amélioration pour un modèle de langage ayant une perplexité de 100 :

1. Améliorations au Niveau des Données :
- Enrichir le corpus d'entraînement avec des données spécifiques au domaine
- Nettoyer et standardiser les données existantes
- Équilibrer la distribution des exemples
- Augmenter la diversité des sources de données
- Intégrer des données annotées de haute qualité

2. Optimisations Techniques :
- Affiner le modèle (fine-tuning) sur des données spécialisées
- Ajuster la température du softmax pour équilibrer créativité et précision
- Implémenter des techniques de régularisation plus sophistiquées
- Optimiser les hyperparamètres du modèle
- Utiliser des techniques d'apprentissage par transfert

3. Améliorations Architecturales :
- Augmenter la capacité du modèle de manière ciblée
- Optimiser les mécanismes d'attention
- Ajouter des couches spécialisées pour le domaine
- Implémenter des architectures hybrides
- Utiliser des techniques de pruning intelligent

4. Stratégies d'Entraînement :
- Adopter un apprentissage curriculum
- Utiliser des techniques d'apprentissage actif
- Implémenter un apprentissage continu
- Optimiser les stratégies de batch
- Utiliser des techniques d'augmentation de données

5. Optimisations Spécifiques au Domaine :
- Intégrer des connaissances du domaine
- Adapter le vocabulaire aux besoins spécifiques
- Développer des métriques d'évaluation personnalisées
- Optimiser pour des cas d'usage particuliers
- Implémenter des règles métier spécifiques

Ces améliorations devraient permettre de réduire significativement la perplexité et d'améliorer les performances globales du modèle.

4) 

Score attribué : 1/5

Justification :
- Syntaxe non naturelle ("comprehend I do not")
- Ordre des mots incorrect en anglais
- Structure de phrase artificielle
- Manque de naturel dans l'expression

Analyse des Problèmes :
- Inversion sujet-verbe inappropriée
- Formalité excessive et mal utilisée
- Manque de contexte dans la réponse
- Ton potentiellement frustrant pour l'utilisateur
- Structure qui trahit une génération artificielle

Version Améliorée : "I apologize, I didn't quite understand your question. Could you please rephrase it?"

Justification des améliorations :
- Structure naturelle de la phrase
- Ton plus professionnel et empathique
- Formulation claire et directe
- Maintien de la politesse sans excès
- Invitation constructive à reformuler




5)

Le code ci dessous implémente :

Détection des Erreurs :
- Analyse des erreurs d'orthographe courantes
- Identification des ambiguïtés
- Détection des pièges temporels

Tests Adversariaux :
- Test d'orthographe (capitol vs capital)
- Test d'ambiguïté (questions avec plusieurs entités)
- Test temporel (contexte historique)

Améliorations Proposées :
- Correction automatique des erreurs courantes
- Ajout de contexte pour clarification
- Gestion des ambiguïtés

Méthodes de Robustesse :
- Normalisation des entrées
- Validation du contexte
- Gestion des cas particuliers

Pour améliorer la robustesse contre ces erreurs, le code propose :
- Correction orthographique automatique
- Ajout de contexte explicite
- Détection des ambiguïtés
- Validation des réponses

In [ ]:

from typing import List, Dict, Tuple
from dataclasses import dataclass
import re

@dataclass
class AdversarialTest:
    prompt: str
    expected_response: str
    potential_errors: List[str]
    test_type: str

class LLMAdversarialTester:
    def __init__(self):
        self.spelling_variants = {
            'capital': ['capitol', 'capitale', 'capitole'],
            'France': ['france', 'Franse', 'Fransse'],
            'Paris': ['paris', 'Parish', 'Париж']
        }
        
    def analyze_spelling_error(self, prompt: str) -> Dict:
        """Analyse les erreurs potentielles d'orthographe"""
        words = prompt.split()
        potential_issues = []
        
        for word in words:
            for correct, variants in self.spelling_variants.items():
                if word.lower() in [v.lower() for v in variants]:
                    potential_issues.append({
                        'incorrect': word,
                        'correct': correct,
                        'type': 'spelling'
                    })
        
        return {
            'prompt': prompt,
            'issues': potential_issues
        }

    def generate_adversarial_tests(self) -> List[AdversarialTest]:
        """Génère une série de tests adversariaux"""
        return [
            AdversarialTest(
                prompt="What is the capitol of France?",
                expected_response="Paris",
                potential_errors=["Confusion orthographique capitol/capital"],
                test_type="spelling"
            ),
            AdversarialTest(
                prompt="Which city is the capital of France... or is it Germany?",
                expected_response="Paris est la capitale de la France. Berlin est la capitale de l'Allemagne.",
                potential_errors=["Confusion due à la question ambiguë"],
                test_type="ambiguity"
            ),
            AdversarialTest(
                prompt="What was the capital of France in 1789?",
                expected_response="Paris",
                potential_errors=["Confusion temporelle historique"],
                test_type="temporal"
            )
        ]

    def improve_prompt_robustness(self, prompt: str) -> str:
        """Améliore la robustesse du prompt"""
        # Correction orthographique
        prompt = self._correct_spelling(prompt)
        # Ajout de contexte
        prompt = self._add_context(prompt)
        return prompt

    def _correct_spelling(self, prompt: str) -> str:
        """Corrige les erreurs d'orthographe courantes"""
        corrected = prompt
        for correct, variants in self.spelling_variants.items():
            for variant in variants:
                corrected = re.sub(
                    rf'\b{variant}\b', 
                    correct, 
                    corrected, 
                    flags=re.IGNORECASE
                )
        return corrected

    def _add_context(self, prompt: str) -> str:
        """Ajoute du contexte pour améliorer la précision"""
        if 'capital' in prompt.lower():
            return f"{prompt} (Please provide the current administrative capital city)"
        return prompt

def main():
    # Création du testeur
    tester = LLMAdversarialTester()
    
    # Analyse du prompt original
    original_prompt = "What is the capitol of France?"
    analysis = tester.analyze_spelling_error(original_prompt)
    
    print("1. Analyse du prompt problématique :")
    print(f"Prompt original : {analysis['prompt']}")
    print("Problèmes détectés :")
    for issue in analysis['issues']:
        print(f"- Erreur : {issue['incorrect']} devrait être {issue['correct']}")
    
    # Amélioration du prompt
    improved_prompt = tester.improve_prompt_robustness(original_prompt)
    print(f"\nPrompt amélioré : {improved_prompt}")
    
    # Génération de tests adversariaux
    print("\n2. Tests adversariaux générés :")
    tests = tester.generate_adversarial_tests()
    for i, test in enumerate(tests, 1):
        print(f"\nTest #{i}")
        print(f"Prompt : {test.prompt}")
        print(f"Réponse attendue : {test.expected_response}")
        print(f"Erreurs potentielles : {', '.join(test.potential_errors)}")
        print(f"Type de test : {test.test_type}")

if __name__ == "__main__":
    main()


1. Analyse du prompt problématique :
Prompt original : What is the capitol of France?
Problèmes détectés :
- Erreur : capitol devrait être capital

Prompt amélioré : What is the capital of France? (Please provide the current administrative capital city)

2. Tests adversariaux générés :

Test #1
Prompt : What is the capitol of France?
Réponse attendue : Paris
Erreurs potentielles : Confusion orthographique capitol/capital
Type de test : spelling

Test #2
Prompt : Which city is the capital of France... or is it Germany?
Réponse attendue : Paris est la capitale de la France. Berlin est la capitale de l'Allemagne.
Erreurs potentielles : Confusion due à la question ambiguë
Type de test : ambiguity

Test #3
Prompt : What was the capital of France in 1789?
Réponse attendue : Paris
Erreurs potentielles : Confusion temporelle historique
Type de test : temporal


In [17]:
# 6)

from typing import List, Dict, Union
import numpy as np
from dataclasses import dataclass
from enum import Enum
import json

class EvaluationMetric(Enum):
    BLEU = "BLEU"
    ROUGE = "ROUGE"
    BERT_SCORE = "BERTScore"
    HUMAN = "Human"
    F1 = "F1"

@dataclass
class MetricAnalysis:
    metric: EvaluationMetric
    strengths: List[str]
    weaknesses: List[str]
    use_cases: List[str]
    score_range: tuple
    complexity: str

class QAEvaluationAnalyzer:
    def __init__(self):
        self.metrics = self._initialize_metrics()
        
    def _initialize_metrics(self) -> Dict[EvaluationMetric, MetricAnalysis]:
        """Initialise l'analyse des différentes métriques"""
        return {
            EvaluationMetric.ROUGE: MetricAnalysis(
                metric=EvaluationMetric.ROUGE,
                strengths=[
                    "Bonne pour évaluer la correspondance de contenu",
                    "Capture bien les informations importantes",
                    "Métrique standard largement utilisée"
                ],
                weaknesses=[
                    "Ne capture pas la cohérence sémantique",
                    "Sensible aux variations de formulation",
                    "Peut manquer des réponses correctes mais différemment formulées"
                ],
                use_cases=[
                    "Évaluation de réponses factuelles",
                    "Comparaison avec des réponses de référence",
                    "Validation de contenu extrait"
                ],
                score_range=(0, 1),
                complexity="Moyenne"
            ),
            
            EvaluationMetric.BERT_SCORE: MetricAnalysis(
                metric=EvaluationMetric.BERT_SCORE,
                strengths=[
                    "Capture la similarité sémantique",
                    "Robuste aux paraphrases",
                    "Utilise des embeddings contextuels"
                ],
                weaknesses=[
                    "Coût computationnel élevé",
                    "Complexité d'implémentation",
                    "Peut être sensible au bruit"
                ],
                use_cases=[
                    "Évaluation de réponses complexes",
                    "Analyse de similarité sémantique",
                    "Validation de cohérence"
                ],
                score_range=(0, 1),
                complexity="Élevée"
            ),
            
            EvaluationMetric.F1: MetricAnalysis(
                metric=EvaluationMetric.F1,
                strengths=[
                    "Simple à comprendre et implémenter",
                    "Équilibre précision et rappel",
                    "Adapté aux réponses courtes"
                ],
                weaknesses=[
                    "Binaire (correct/incorrect)",
                    "Ne capture pas les nuances",
                    "Limité pour les réponses complexes"
                ],
                use_cases=[
                    "QA factuel",
                    "Extraction d'informations simples",
                    "Validation rapide"
                ],
                score_range=(0, 1),
                complexity="Faible"
            )
        }

    def compare_metrics(self) -> Dict:
        """Compare les différentes métriques"""
        comparison = {}
        for metric, analysis in self.metrics.items():
            comparison[metric.value] = {
                "forces": analysis.strengths,
                "faiblesses": analysis.weaknesses,
                "cas_utilisation": analysis.use_cases,
                "complexité": analysis.complexity
            }
        return comparison

    def recommend_metric(self, task_type: str) -> Dict:
        """Recommande la meilleure métrique selon le type de tâche"""
        recommendations = {
            "factuel": EvaluationMetric.F1,
            "complexe": EvaluationMetric.BERT_SCORE,
            "extractif": EvaluationMetric.ROUGE
        }
        
        recommended = recommendations.get(task_type, EvaluationMetric.BERT_SCORE)
        return {
            "métrique_recommandée": recommended.value,
            "justification": self._get_recommendation_justification(recommended, task_type)
        }

    def _get_recommendation_justification(self, metric: EvaluationMetric, task_type: str) -> str:
        """Génère une justification pour la recommandation"""
        justifications = {
            EvaluationMetric.F1: "Optimal pour les réponses factuelles courtes nécessitant une évaluation précise",
            EvaluationMetric.BERT_SCORE: "Meilleur pour les réponses complexes nécessitant une compréhension sémantique",
            EvaluationMetric.ROUGE: "Adapté pour l'évaluation de contenu extrait et la correspondance textuelle"
        }
        return justifications.get(metric, "Justification non disponible")

def main():
    analyzer = QAEvaluationAnalyzer()
    
    print("Analyse Comparative des Métriques d'Évaluation pour QA")
    print("\n1. Comparaison des Métriques :")
    comparison = analyzer.compare_metrics()
    
    for metric, details in comparison.items():
        print(f"\n{metric}")
        print("Forces :")
        for strength in details["forces"]:
            print(f"  - {strength}")
        print("Faiblesses :")
        for weakness in details["faiblesses"]:
            print(f"  - {weakness}")
        print(f"Complexité : {details['complexité']}")
    
    print("\n2. Recommandations par Type de Tâche :")
    task_types = ["factuel", "complexe", "extractif"]
    for task in task_types:
        recommendation = analyzer.recommend_metric(task)
        print(f"\nPour QA {task} :")
        print(f"Métrique recommandée : {recommendation['métrique_recommandée']}")
        print(f"Justification : {recommendation['justification']}")

if __name__ == "__main__":
    main()


Analyse Comparative des Métriques d'Évaluation pour QA

1. Comparaison des Métriques :

ROUGE
Forces :
  - Bonne pour évaluer la correspondance de contenu
  - Capture bien les informations importantes
  - Métrique standard largement utilisée
Faiblesses :
  - Ne capture pas la cohérence sémantique
  - Sensible aux variations de formulation
  - Peut manquer des réponses correctes mais différemment formulées
Complexité : Moyenne

BERTScore
Forces :
  - Capture la similarité sémantique
  - Robuste aux paraphrases
  - Utilise des embeddings contextuels
Faiblesses :
  - Coût computationnel élevé
  - Complexité d'implémentation
  - Peut être sensible au bruit
Complexité : Élevée

F1
Forces :
  - Simple à comprendre et implémenter
  - Équilibre précision et rappel
  - Adapté aux réponses courtes
Faiblesses :
  - Binaire (correct/incorrect)
  - Ne capture pas les nuances
  - Limité pour les réponses complexes
Complexité : Faible

2. Recommandations par Type de Tâche :

Pour QA factuel :
Métriqu